# Title

## Introduction

### Load Data into R

In [32]:
library(tidyverse)
library(repr)
library(readxl)
library(stringr)
options(repr.matrix.max.rows = 6)

In [95]:
data <- read_csv("https://raw.githubusercontent.com/SarahHoang/dsci-100-project-group-162/main/spambase.data", 
                      col_names = FALSE)
data

Rows: 4601 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (58): X1, X2, X3, X4, X5, X6, X7, X8, X9, X10, X11, X12, X13, X14, X15, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.000,0,0.778,0.000,0.000,3.756,61,278,1
0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,⋯,0.00,0.132,0,0.372,0.180,0.048,5.114,101,1028,1
0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,⋯,0.01,0.143,0,0.276,0.184,0.010,9.821,485,2259,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.30,0,0.30,0,0.00,0,0,0,0,0,⋯,0.102,0.718,0,0.000,0,0,1.404,6,118,0
0.96,0,0.00,0,0.32,0,0,0,0,0,⋯,0.000,0.057,0,0.000,0,0,1.147,5,78,0
0.00,0,0.65,0,0.00,0,0,0,0,0,⋯,0.000,0.000,0,0.125,0,0,1.250,5,40,0


In [122]:
column_names <- read_delim("https://raw.githubusercontent.com/SarahHoang/dsci-100-project-group-162/main/spambase.names", 
                         skip = 33,
                        col_names = FALSE) |> 
                        select(X1) |>
                        rename("col_name" = X1)

col <- toString(column_names)
col

colnames(email_data) <- c("word_freq_make", "word_freq_address", "word_freq_all", "word_freq_3d", "word_freq_our", "word_freq_over", 
  "word_freq_remove", "word_freq_internet", "word_freq_order", "word_freq_mail", "word_freq_receive", "word_freq_will", 
  "word_freq_people", "word_freq_report", "word_freq_addresses", "word_freq_free", "word_freq_business", "word_freq_email", 
  "word_freq_you", "word_freq_credit", "word_freq_your", "word_freq_font", "word_freq_000", "word_freq_money", 
  "word_freq_hp", "word_freq_hpl", "word_freq_george", "word_freq_650", "word_freq_lab", "word_freq_labs", 
  "word_freq_telnet", "word_freq_857", "word_freq_data", "word_freq_415", "word_freq_85", "word_freq_technology", 
  "word_freq_1999", "word_freq_parts", "word_freq_pm", "word_freq_direct", "word_freq_cs", "word_freq_meeting", 
  "word_freq_original", "word_freq_project", "word_freq_re", "word_freq_edu", "word_freq_table", "word_freq_conference", 
  "char_freq_;", "char_freq_(", "char_freq_[", "char_freq_!", "char_freq_$", "char_freq_#", "capital_run_length_average", 
  "capital_run_length_longest", "capital_run_length_total", "Spam or non-spam")

Rows: 57 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: ":"
chr (2): X1, X2

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "c(\"word_freq_make\", \"word_freq_address\", \"word_freq_all\", \"word_freq_3d\", \"word_freq_our\", \"word_freq_over\", \"word_freq_remove\", \"word_freq_internet\", \"word_freq_order\", \"word_freq_mail\", \"word_freq_receive\", \"word_freq_will\", \"word_freq_people\", \"word_freq_report\", \"word_freq_addresses\", \"word_freq_free\", \"word_freq_business\", \"word_freq_email\", \"word_freq_you\", \"word_freq_credit\", \"word_freq_your\", \"word_freq_font\", \"word_freq_000\", \"word_freq_money\", \"word_freq_hp\", \"word_freq_hpl\", \"word_freq_george\", \n\"word_freq_650\", \"word_freq_lab\", \"word_freq_labs\", \"word_freq_telnet\", \"word_freq_857\", \"word_freq_data\", \"word_freq_415\", \"word_freq_85\", \"word_freq_technology\", \"word_freq_1999\", \"word_freq_parts\", \"word_freq_pm\", \"word_freq_direct\", \"word_freq_cs\", \"word_freq_meeting\", \"word_freq_original\", \"word_freq_project\", \"word_freq_re\", \"word_freq_edu\", \"word_freq_table\", \"word_freq_conference\", \"char_freq_;\", \"char_freq_(\", \"char_freq_[\", \"char_freq_!\", \"char_freq_$\", \"char_freq_#\", \"capital_run_length_average\", \n\"capital_run_length_longest\", \"capital_run_length_total\")"

In [121]:
email_data

word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,⋯,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,Spam or non-spam
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.00,0.64,0.64,0,0.32,0.00,0.00,0.00,0.00,0.00,⋯,0.00,0.000,0,0.778,0.000,0.000,3.756,61,278,1
0.21,0.28,0.50,0,0.14,0.28,0.21,0.07,0.00,0.94,⋯,0.00,0.132,0,0.372,0.180,0.048,5.114,101,1028,1
0.06,0.00,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,⋯,0.01,0.143,0,0.276,0.184,0.010,9.821,485,2259,1
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
0.30,0,0.30,0,0.00,0,0,0,0,0,⋯,0.102,0.718,0,0.000,0,0,1.404,6,118,0
0.96,0,0.00,0,0.32,0,0,0,0,0,⋯,0.000,0.057,0,0.000,0,0,1.147,5,78,0
0.00,0,0.65,0,0.00,0,0,0,0,0,⋯,0.000,0.000,0,0.125,0,0,1.250,5,40,0


## Methods

## Expected outcomes and significance